# Daily Fantasy Football Projection comparison

In [135]:
import numpy as np
import pandas as pd
import requests

from pymongo import MongoClient
import pprint

# Beautiful Soup parses HTML documents in python.
from bs4 import BeautifulSoup
import json
import time

import matplotlib.pyplot as plt
# from matplotlib import pyplot as plt

%matplotlib inline
plt.style.use('ggplot')


## Read html


In [136]:
df = pd.read_html('https://www.footballdiehards.com/fantasyfootball/dailygames/FanDuel-Salary-data.cfm', header = 1)

# Turn list into DatFrame
salary = df[0].dropna(axis=0, thresh=4)

In [137]:
salary

sort_sal = salary.sort_values(by = 'SALARY', ascending = True)

sort_sal

,Player,Pos,year,week,SALARY,Score,Factor,Rank
30,"Henry, Derrick",RB,2020,13,$10000,19.3,1.9,3
12,"Cook, Dalvin",RB,2020,13,$10500,23.0,2.2,1
117,Jacksonville,D,2020,13,$3200,4.3,1.4,30
106,New York Jets,D,2020,13,$3300,4.7,1.4,24
120,Philadelphia,D,2020,13,$3400,4.6,1.3,27
...,...,...,...,...,...,...,...,...
16,"Chubb, Nick",RB,2020,13,$8700,18.2,2.1,5
33,"Murray, Kyler",QB,2020,13,$8700,16.4,1.9,8
11,"Wilson, Russell",QB,2020,13,$9000,19.8,2.2,2
1,"Mahomes, Patrick",QB,2020,13,$9000,22.5,2.5,1


In [138]:
#renaming player name column to just name
salary = salary.rename(columns={'Player': 'name'})

# reversing name to combine with player projection data
salary['name'] = salary['name'].str.replace(r'(.+),\s+(.+)', r'\2 \1')

#Converting SALARY to a float type and dropping $

salary['SALARY'] = salary['SALARY'].str.replace('$', '')
salary['SALARY'] = salary['SALARY'].astype('float')

# importing and cleaning QB df

In [139]:

#Reading in QB DF
qb = pd.read_csv('Fantasy_proj/qb.csv')
qb = qb.dropna()


# drop row 2020 OUTLOOK column that just contains string
qb = qb.drop(['2020 OUTLOOK'], axis = 1)




In [140]:
#rename the Player Name row to match Salary df
qb = qb.rename(columns={'PLAYER NAME': 'name'})
qb = qb.rename(columns={'PLAYER POSITION': 'Pos'})

#drop pointless row
#qb= qb.drop(['Unnamed: 0'], axis = 1)

type(qb['name'])

pandas.core.series.Series

In [141]:
#create a single dataframe with QBs from salary added to QB for fantasy projections

qb_sal = pd.merge(qb, salary, how='left', left_on='name', right_on='name')

#qb_sal


In [142]:
#Leaving Out Rows Where SALARY = NaN = Means a Player is not playing this week

qb_sal = qb_sal[qb_sal['SALARY'].notna()]

#qb_sal

In [143]:
# Filtered out more rows where player Score does not == '-', 
# Also mean they wont get much playing time or wont play at all


qb_sal =qb_sal[qb_sal.Score != '-']

#convert $ value into float in 2 step first replace $ then convert to float

# qb_sal['SALARY'] = qb_sal['SALARY'].str.replace('$', '')
# qb_sal['SALARY'] = qb_sal['SALARY'].astype('float')


#Final qb list including their salary filtered to only players playing
qb_sal

,Unnamed: 0,name,TEAM NAME,Pos_x,2019 COMPLETED PASS,2019 ATTEMPTED PASS,2019 PASSING YDS,2019 PASSING TD,2019 INT,2019 CAR,...,2020 RUSH YDS,2020 RUSH TD,2020 FPTS,Pos_y,year,week,SALARY,Score,Factor,Rank
1,2,Patrick Mahomes,Chiefs,QB,319.0,484.0,4031.0,26.0,5.0,43.0,...,280.0,3.0,332.66,QB,2020.0,13.0,9000.0,22.5,2.5,1.0
2,3,Deshaun Watson,Texans,QB,333.0,495.0,3852.0,26.0,12.0,82.0,...,519.0,4.0,314.06,QB,2020.0,13.0,8200.0,18.2,2.2,5.0
4,5,Russell Wilson,Seahawks,QB,341.0,516.0,4110.0,31.0,5.0,75.0,...,397.0,2.0,293.96,QB,2020.0,13.0,9000.0,19.8,2.2,2.0
5,6,Kyler Murray,Cardinals,QB,349.0,542.0,3722.0,20.0,12.0,93.0,...,469.0,3.0,293.94,QB,2020.0,13.0,8700.0,16.4,1.9,8.0
7,8,Matt Ryan,Falcons,QB,408.0,616.0,4466.0,26.0,14.0,34.0,...,125.0,1.0,282.50,QB,2020.0,13.0,7400.0,13.1,1.8,16.0
8,9,Matthew Stafford,Lions,QB,187.0,291.0,2499.0,19.0,5.0,20.0,...,123.0,1.0,268.80,QB,2020.0,13.0,6800.0,10.9,1.6,21.0
9,10,Carson Wentz,Eagles,QB,388.0,607.0,4039.0,27.0,7.0,62.0,...,213.0,2.0,268.38,QB,2020.0,13.0,7300.0,11.4,1.6,20.0
11,12,Josh Allen,Bills,QB,271.0,461.0,3089.0,20.0,9.0,109.0,...,545.0,5.0,266.26,QB,2020.0,13.0,8600.0,18.5,2.2,4.0
12,13,Aaron Rodgers,Packers,QB,353.0,569.0,4002.0,26.0,4.0,46.0,...,206.0,1.0,265.79,QB,2020.0,13.0,8500.0,18.8,2.2,3.0
13,14,Daniel Jones,Giants,QB,284.0,459.0,3027.0,24.0,12.0,45.0,...,267.0,2.0,254.75,QB,2020.0,13.0,7100.0,5.0,0.7,36.0


# Cleaning RB DATA

In [144]:
#importing rb fantasy projection & dropping NA
rb = pd.read_csv('Fantasy_proj/rb.csv')
rb = rb.rename(columns={'PLAYER NAME': 'name'})
#rb.dropna()

In [145]:
rb_sal = pd.merge(rb, salary, how='left', left_on='name', right_on='name')

In [146]:
#Drop columns with meaningless values and leave out rows where Salary == NaN values

rb_sal.drop(['Pos', 'year', 'week'], axis=1)
rb_sal = rb_sal[rb_sal['SALARY'].notna()]
# drop rows where Score = '-' Means they prob not playing
rb_sal =rb_sal[rb_sal.Score != '-']

#drop rows where TEAM NAME = FA (Free Agent Prob Not Playing)
rb_sal =rb_sal[rb_sal['TEAM NAME'] != 'FA']


#drop rows where Salary > certain threshold ... Means they prob wont be playing or very little playing time

#rb_sal = rb_sal[rb_sal['SALARY'] > 5000]

In [147]:
#Leave out rows where Salary is below 5000 threshold== Means they wont get much playing time

rb_sal = rb_sal[rb_sal['SALARY'] > 5000]



# Cleaning WR df

In [148]:
#importing WR projection data and changing name column to combine DF
wr = pd.read_csv('Fantasy_proj/wr.csv')
wr= wr.rename(columns={'PLAYER NAME': 'name'})

In [149]:
#merging WR projections with SALARY

wr_sal = pd.merge(wr, salary, how='left', left_on='name', right_on='name')



In [152]:
#Drop columns with meaningless values and leave out rows where Salary == NaN values

wr_sal.drop(['Pos', 'year', 'week'], axis=1)
wr_sal = wr_sal[wr_sal['SALARY'].notna()]
# drop rows where Score = '-' Means they prob not playing
wr_sal =wr_sal[wr_sal.Score != '-']

#drop rows where TEAM NAME = FA (Free Agent Prob Not Playing)
wr_sal =wr_sal[wr_sal['TEAM'] != 'FA']

wr_sal = wr_sal[wr_sal['SALARY'] > 5000]

In [154]:
#wr_sal

# Cleaning TE Data

In [163]:
#importing and changing name column to combine with salary df

te = pd.read_csv('Fantasy_proj/te.csv')
te = te.rename(columns={'PLAYER NAME': 'name'})

In [166]:
#merging te projections with SALARY

te_sal = pd.merge(te, salary, how='left', left_on='name', right_on='name')

te_sal

,Unnamed: 0,name,TEAM NAME,PLAYER POSITION,2019 TAR,2019 REC,2019 REC YDS,2019 AVG REC YDS,2019 REC TD,2019 RUSH CAR,...,2020 RUSH TD,2020 FPTS,2020 OUTLOOK,Pos,year,week,SALARY,Score,Factor,Rank
0,1,Travis Kelce,Chiefs,TE,136.0,97.0,1229.0,12.7,5.0,1.0,...,0.0,251.34,Kelce enters the 2020 season attempting to pac...,TE,2020.0,13.0,8000.0,12.5,1.6,1.0
1,2,George Kittle,49ers,TE,107.0,85.0,1053.0,12.4,5.0,5.0,...,0.0,231.77,Kittle wasn't much of a factor as a pass-catch...,TE,2020.0,13.0,6000.0,-,-,999.0
2,3,Zach Ertz,Eagles,TE,135.0,88.0,916.0,10.4,6.0,0.0,...,0.0,201.16,Ertz had his hands full playing through injuri...,TE,2020.0,13.0,5700.0,4.3,0.8,23.0
3,4,Mark Andrews,Ravens,TE,98.0,64.0,852.0,13.3,10.0,0.0,...,0.0,199.44,"A 2018 third-round pick, Andrews exited his ro...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,Darren Waller,Raiders,TE,117.0,90.0,1145.0,12.7,3.0,2.0,...,0.0,192.05,Waller's breakout 2019 season marked one of th...,TE,2020.0,13.0,7000.0,10.7,1.5,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
186,187,Ben Watson,Patriots,TE,24.0,17.0,173.0,10.2,0.0,0.0,...,NaN,0.00,No outlook available.,NaN,NaN,NaN,NaN,NaN,NaN,NaN
187,188,Delanie Walker,Titans,TE,31.0,21.0,215.0,10.2,2.0,0.0,...,NaN,0.00,No outlook available.,NaN,NaN,NaN,NaN,NaN,NaN,NaN
188,189,C.J. Conrad,FA,TE,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.00,No outlook available.,NaN,NaN,NaN,NaN,NaN,NaN,NaN
189,190,Bradley Sowell,FA,TE,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,0.00,No outlook available.,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [167]:
#Drop columns with meaningless values and leave out rows where Salary == NaN values

te_sal.drop(['Pos', 'year', 'week'], axis=1)
te_sal = te_sal[te_sal['SALARY'].notna()]
# drop rows where Score = '-' Means they prob not playing
te_sal =te_sal[te_sal.Score != '-']

#drop rows where TEAM NAME = FA (Free Agent Prob Not Playing)
te_sal =te_sal[te_sal['TEAM NAME'] != 'FA']

te_sal = te_sal[te_sal['SALARY'] > 5000]

# importing and cleaning kicker data

In [178]:
#no kicker salary no kicker in draft Kings
kicker = pd.read_csv('Fantasy_proj/kicker.csv')
kicker = kicker.rename(columns={'PLAYER NAME': 'name'})



#Merging Salary and kicker DF

In [190]:
#no kicker salary no kicker in draft Kings
defense = pd.read_csv('Fantasy_proj/defense.csv')
defense = defense.rename(columns={'PLAYER NAME': 'name'})

In [192]:
defense = pd.merge(defense, salary, how='left', left_on='name', right_on='name')

KeyError: 'name'

,Unnamed: 0,PLAYER POSITION,TEAM NAME,2019 COMPLETED 1-39,2019 ATTEMPTED 1-39,2019 COMPLETED 40-49,2019 ATTEMPTED 40-49,2019 COMPLETED 50+,2019 ATTEMPTED 50+,2019 COMPLETED TOT,...,Factor_x,Rank_x,name,Pos_y,year_y,week_y,SALARY_y,Score_y,Factor_y,Rank_y


,name,Pos,year,week,SALARY,Score,Factor,Rank
0,Austin Ekeler,RB,2020,13,7000.0,21.8,3.1,2
1,Patrick Mahomes,QB,2020,13,9000.0,22.5,2.5,1
2,Antonio Gibson,RB,2020,13,6700.0,16.6,2.5,8
3,David Montgomery,RB,2020,13,6200.0,15.1,2.4,11
4,James Robinson,RB,2020,13,7800.0,18.4,2.4,4
...,...,...,...,...,...,...,...,...
642,C.J. Beathard,QB,2020,13,6200.0,-,-,999
643,Matt Barkley,QB,2020,13,6100.0,-,-,999
644,Brett Rypien,QB,2020,13,6100.0,-,-,999
645,Jeff Driskel,QB,2020,13,6100.0,-,-,999


In [126]:
#Completed data frames Titles

#qb_sal
#rb_sal
#wr_sal
#te_sal


SyntaxError: invalid syntax (<ipython-input-126-46ec41c0fb5d>, line 1)